In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor


## Data traitment

Collecte et traitement des données

In [2]:
# Collecte des données
X= pd.read_csv(r"X_train_filled.csv",index_col=0)
Y = pd.read_csv(r"C:\Users\collo\Documents\Etudes\Dauphine\Machine Learning\Projet\DAUPHINE_Machine_Learning\y_train.csv",index_col=0)
data_train = pd.merge(X, Y, left_index=True, right_index=True)
data_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,spot_id_delta
DELIVERY_START,,,,,,,,,,
2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,116.808914,-36.874770
2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,116.345711,-12.643588
2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,108.858177,-1.950193
2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,112.760529,1.938272
2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,114.032677,0.199907
...,...,...,...,...,...,...,...,...,...,...
2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.110000,6.029303
2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.660000,13.576177
2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.010000,17.478945


In [3]:
# Traitement
## Missing Data
data = data_train.dropna(axis=1, thresh=len(data_train)*0.25)
data = data.dropna()
#data = data.ffill()
X = data[[col for col in data.columns if col !='spot_id_delta']]
Y = data['spot_id_delta'].to_numpy()
## Format
X = pd.get_dummies(X)
## Split
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

## Model Selection

In [4]:
scalers = [None,StandardScaler(), MinMaxScaler(), RobustScaler()]
models = {
    'XGBoost' : XGBRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet': ElasticNet(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    #'K-Neighbors Regression': KNeighborsRegressor()
    }

res = list()
for mod in models.values() :
    print(f"Model: {mod.__class__.__name__}")
    for scaler in scalers:
        # Appliquer le scaler
        if scaler is None :
            X_train_scaled = X_train
            X_test_scaled = X_test
        else :
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
        # Initialiser le modèle
        np.random.RandomState(42)
        model = mod
        
        # Entraîner le modèle de base
        model.fit(X_train_scaled, y_train)
        
        # Évaluation du modèle
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Validation croisée
        cv_scores = sk.model_selection.cross_val_score(model, X_train_scaled, y_train, cv=10, scoring='neg_mean_squared_error')
        res.append([mod.__class__.__name__,scaler.__class__.__name__,mse,r2,cv_scores])


Model: XGBRegressor
Model: LinearRegression
Model: Ridge
Model: Lasso
Model: ElasticNet
Model: SVR
Model: DecisionTreeRegressor
Model: RandomForestRegressor
Model: GradientBoostingRegressor


In [5]:
res = pd.DataFrame(res,columns=["model","scaler","mse","r2","cv_score"]).sort_values(by="r2",ascending=False)
res["cv_mean"] = res['cv_score'].apply(lambda x : np.mean(x))
res

,model,scaler,mse,r2,cv_score,cv_mean
29,RandomForestRegressor,StandardScaler,349.016851,0.582601,"[-2553.268676519713, -304.6412013821945, -280....",-904.884295
30,RandomForestRegressor,MinMaxScaler,352.708588,0.578186,"[-2473.7869654094075, -315.2835624236182, -287...",-878.434580
28,RandomForestRegressor,NoneType,358.271795,0.571533,"[-2384.8293939114096, -316.89763907602605, -29...",-870.655084
31,RandomForestRegressor,RobustScaler,359.138491,0.570497,"[-2534.303373925694, -309.40281233548234, -294...",-891.461474
0,XGBRegressor,NoneType,418.388251,0.499638,"[-1654.1415964049188, -355.9444930622808, -347...",-764.747954
1,XGBRegressor,StandardScaler,418.388251,0.499638,"[-1654.1415964049188, -355.9444930622808, -347...",-764.648968
2,XGBRegressor,MinMaxScaler,418.388251,0.499638,"[-1654.1415964049188, -355.9444930622808, -347...",-764.648968
3,XGBRegressor,RobustScaler,418.388251,0.499638,"[-1654.1415964049188, -355.9444930622808, -347...",-764.648968
32,GradientBoostingRegressor,NoneType,554.608074,0.336729,"[-3065.9865589703904, -452.7250522104862, -440...",-1235.997158
33,GradientBoostingRegressor,StandardScaler,555.471741,0.335697,"[-3077.873909122212, -452.992880704556, -440.3...",-1244.440185


## Model

Optimisation des paramètres

Creation et Validation du modèle

In [6]:
model = RandomForestRegressor(random_state=42) #**best_params)

# Entraîner le modèle sur les données d'entraînement complètes
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

Mean Squared Error: 348.3053811133128
R2 Score: 0.5834523006241926


In [7]:
model.fit(X_test,y_test)

RandomForestRegressor(random_state=42)

On utilie le model sur les valeurs manquantes

In [8]:
X_test = pd.read_csv(r"X_test_filled.csv",index_col=0)
X_test

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
DELIVERY_START,,,,,,,,,
2023-04-02 00:00:00+02:00,45814.0,3386.0,10902.0,36705.0,6359.0,0.0,56.764535,0.0,111.145187
2023-04-02 01:00:00+02:00,44084.0,3386.0,10902.0,36705.0,6469.0,0.0,54.262133,0.0,110.423241
2023-04-02 02:00:00+02:00,43281.0,3386.0,10902.0,36705.0,6511.0,0.0,78.105928,0.0,110.569878
2023-04-02 03:00:00+02:00,40825.0,3386.0,10902.0,36705.0,6628.0,0.0,78.187557,0.0,82.747284
2023-04-02 04:00:00+02:00,39181.0,3386.0,10902.0,36705.0,6700.0,0.0,96.765484,0.0,82.842125
...,...,...,...,...,...,...,...,...,...
2023-10-24 19:00:00+02:00,49686.0,2226.0,11749.0,42980.0,4901.0,0.0,247.887323,0.0,125.670000
2023-10-24 20:00:00+02:00,53397.0,2226.0,11749.0,42980.0,5584.0,0.0,343.192642,0.0,139.580000
2023-10-24 21:00:00+02:00,50586.0,2226.0,11749.0,42980.0,6306.0,0.0,471.875973,0.0,147.930000


In [9]:
X_test = pd.get_dummies(X_test)
y_test = model.predict(X_test)

In [10]:
Y_test = pd.Series(y_test,index = X_test.index)
Y_test.name = "spot_id_delta"
Y_test

DELIVERY_START
2023-04-02 00:00:00+02:00    -1.571948
2023-04-02 01:00:00+02:00    -0.780328
2023-04-02 02:00:00+02:00     0.553251
2023-04-02 03:00:00+02:00    -6.187166
2023-04-02 04:00:00+02:00    -4.472275
                               ...    
2023-10-24 19:00:00+02:00   -16.524816
2023-10-24 20:00:00+02:00    -6.019763
2023-10-24 21:00:00+02:00    -4.085833
2023-10-24 22:00:00+02:00     1.541472
2023-10-24 23:00:00+02:00     2.869137
Name: spot_id_delta, Length: 4942, dtype: float64

In [11]:
Y_test.to_csv("y_test.csv")